In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
df = pd.DataFrame(pd.read_excel('./datasci/cleanedup/whl_2025_base.xlsx'))
df.head()

In [ ]:
'''
IDENTIFIERS:
- game_id
- record_id

ENTITIES:
- home_team
- away_team
- home_goalie
- away_goalie

CONTEXT:
- home_off_line
- away_off_line
- home_def_pairing
- away_def_pairing
- went_ot

OUTCOMES:
- home_goals
- away_goals
- home_shots
- away_shots
- home_penalties_committed
- away_penalties_committed

DERIVED METRICS:
- home_xg
- away_xg
- home_max_xg
- away_max_xg
'''

In [ ]:
sum_cols = [
    "home_goals", "away_goals",
    "home_shots", "away_shots",
    "home_xg", "away_xg",
    "home_assists", "away_assists",
    "home_penalties_committed", "away_penalties_committed",
    "home_penalty_minutes", "away_penalty_minutes"
]
first_cols = [
    "home_team",
    "away_team",
    "went_ot"
]
agg_dict = {}

for col in sum_cols:
    agg_dict[col] = "sum"

for col in first_cols:
    agg_dict[col] = "first"
games = (
    df
    .groupby("game_id", as_index=False)
    .agg(agg_dict)
)
games.shape
games["home_score"] = games["home_goals"]
games["away_score"] = games["away_goals"]

games["goal_diff"] = games["home_score"] - games["away_score"]
games["total_goals"] = games["home_score"] + games["away_score"]

games["shot_diff"] = games["home_shots"] - games["away_shots"]
games["total_shots"] = games["home_shots"] + games["away_shots"]

games

home_games = games.copy()

home_games["team"] = home_games["home_team"]
home_games["opponent"] = home_games["away_team"]

home_games["goals_for"] = home_games["home_score"]
home_games["goals_against"] = home_games["away_score"]

home_games["shots_for"] = home_games["home_shots"]
home_games["shots_against"] = home_games["away_shots"]

home_games["xg_for"] = home_games["home_xg"]
home_games["xg_against"] = home_games["away_xg"]

home_games["is_home"] = 1
away_games = games.copy()

away_games["team"] = away_games["away_team"]
away_games["opponent"] = away_games["home_team"]

away_games["goals_for"] = away_games["away_score"]
away_games["goals_against"] = away_games["home_score"]

away_games["shots_for"] = away_games["away_shots"]
away_games["shots_against"] = away_games["home_shots"]

away_games["xg_for"] = away_games["away_xg"]
away_games["xg_against"] = away_games["home_xg"]

away_games["is_home"] = 0
team_games = pd.concat([home_games, away_games], ignore_index=True)
team_games["goal_diff"] = team_games["goals_for"] - team_games["goals_against"]

team_games["win"] = (team_games["goal_diff"] > 0).astype(int)
team_games["loss"] = (team_games["goal_diff"] < 0).astype(int)
team_season = (
    team_games
    .groupby("team", as_index=False)
    .agg(
        games_played=("team", "count"),
        wins=("win", "sum"),
        losses=("loss", "sum"),
        goals_for=("goals_for", "sum"),
        goals_against=("goals_against", "sum"),
        shots_for=("shots_for", "sum"),
        shots_against=("shots_against", "sum"),
        xg_for=("xg_for", "sum"),
        xg_against=("xg_against", "sum"),
        avg_goal_diff=("goal_diff", "mean"),
        home_games=("is_home", "sum")
    )
)
team_season["goals_per_game"] = team_season["goals_for"] / team_season["games_played"]
team_season["goals_against_per_game"] = team_season["goals_against"] / team_season["games_played"]

team_season["shot_diff"] = team_season["shots_for"] - team_season["shots_against"]
team_season["xg_diff"] = team_season["xg_for"] - team_season["xg_against"]
team_season["win_pct"] = team_season["wins"] / team_season["games_played"]
team_season["loss_pct"] = team_season["losses"] / team_season["games_played"]

team_season
team_season.sort_values("loss_pct", ascending=False).head()

home_wins = games[games['home_goals'] > games['away_goals']]
home_wins.describe()
'''
# TODO: Calculate 'Home Multiplier' by comparing Home xG/60 vs Away xG/60 across the league.
# TODO: Apply a 'Neutrality Filter' to penalize home-heavy schedules in the rankings.

# --- LOGIC FOR went_ot (The Volatility Filter) ---
# 1. Regulation Performance: Use this to isolate 'Regulation Goal Differential'.
#    A team winning 5-0 in regulation is significantly stronger than a team
#    winning 1-0 in OT. The former shows dominance; the latter shows a coin-flip.
#
# 2. 'The Paper Tiger' Check: Identify teams with high standings but high OT win rates.
#    If a team relies on OT/Shootouts, their Power Ranking should be ADJUSTED DOWN
#    as OT results are less repeatable than 5-on-5 play.
#
# 3. 'The Resilience' Factor: Boost teams with high OT Loss counts.
#    In the standings, they look like losers (0 wins), but in reality,
#    they are competitive enough to hold elite teams to a draw for 60 minutes.
#
# 4. Usage Normalization: Since OT adds extra 'toi', always use 'per 60 minutes'
#    rates (e.g., xG/60) to ensure OT minutes don't artificially inflate total stats.

# --- LOGIC FOR home_off_line (The Roster Strength Factor) ---
# 1. Roster Depth: Compare 'first_off' vs 'second_off' xG/60.
#    - 'One-Line Wonders': Teams with a huge drop-off in quality (e.g., 1st line 3.0 xG, 2nd line 0.5 xG).
#    - 'Balanced Giants': Teams where both lines produce consistently.
#    ACTION: Reward 'Balanced' teams with a higher stability score in rankings.
#
# 2. Situational Power: Isolate 'PP_up' (Power Play) records.
#    - Standing might be low, but if 'PP_up' xG/60 is top 5, they are a 'Danger Team'.
#    ACTION: Add a 'Special Teams Grade' to the final Power Ranking.
#
# 3. 5-on-5 Purity: Filter for 'first_off' and 'second_off' only to find 'Even-Strength' dominance.
#    - This is the most repeatable part of hockey.
#    ACTION: Use Even-Strength xG Differential as 50% of the total Power Ranking weight.
#
# 4. Tactical Matchups: Link with 'away_def_pairing' to see which lines 'crush' weaker defenders.
#    - Identify teams that successfully hunt mismatches (e.g., first_off vs. opponent's second_def).

# --- LOGIC FOR home_def_pairing (The Shutdown Metric) ---
# 1. Shutdown Quality: Calculate 'xG Allowed per 60' for each pairing.
#    A team's 'Defensive Rank' should be heavily weighted by the first_def unit.
#
# 2. Defensive Depth: Measure the 'Reliability Gap' between 1st and 2nd pairs.
#    Teams with a strong second_def are 'Tournament Hardened' and harder to exploit.
#
# 3. PK Specialist Rank: Filter for 'PP_kill_dwn'.
#    Identify teams that effectively suppress xG even when man-down.
#    High PK efficiency is a major signal for 'Playoff Ready' power rankings.
#
# 4. Goal-Save Delta: Compare 'Actual Goals Allowed' vs 'xG Allowed' per pairing.
#    If a pairing allows high xG but zero goals, the goalie is 'bailing them out'.




'''
home_ot = games[['home_team', 'went_ot']].rename(columns={'home_team': 'team'})
away_ot = games[['away_team', 'went_ot']].rename(columns={'away_team': 'team'})

# Combine both lists and sum the OT occurrences
team_ot_counts = pd.concat([home_ot, away_ot]).groupby('team')['went_ot'].sum().reset_index()
team_ot_counts.columns = ['team', 'ot_games_count']

# Sort by the number of OT games
team_ot_counts = team_ot_counts.sort_values(by='ot_games_count', ascending=False)
print(team_ot_counts)
team_games["reg_win"] = ((team_games["goal_diff"] > 0) & (team_games["went_ot"] == 0)).astype(int)
team_games["ot_win"] = ((team_games["goal_diff"] > 0) & (team_games["went_ot"] == 1)).astype(int)
team_games["ot_loss"] = ((team_games["goal_diff"] < 0) & (team_games["went_ot"] == 1)).astype(int)
team_games["reg_loss"] = ((team_games["goal_diff"] < 0) & (team_games["went_ot"] == 0)).astype(int)
team_season = (
    team_games
    .groupby("team", as_index=False)
    .agg(
        games_played=("team", "count"),
        reg_wins=("reg_win", "sum"),
        ot_wins=("ot_win", "sum"),
        ot_losses=("ot_loss", "sum"),
        reg_losses=("reg_loss", "sum"),
        goals_for=("goals_for", "sum"),
        goals_against=("goals_against", "sum"),
        xg_for=("xg_for", "sum"),
        xg_against=("xg_against", "sum"),
        total_ot_games=("went_ot", "sum")
    )
)

# 3. Calculate the New Metrics (The Logic Check)
# Regulation Performance (Pure Dominance)
team_season["reg_win_pct"] = team_season["reg_wins"] / team_season["games_played"]

# 'The Paper Tiger' Check (What % of their wins are 'coin-flips'?)
# High ratio = Adjusted Down
team_season["ot_reliance_ratio"] = team_season["ot_wins"] / (team_season["reg_wins"] + team_season["ot_wins"] + 1e-6)

# 'The Resilience' Factor (High OT losses = Competitiveness)
# High ratio = Adjusted Up
team_season["resilience_factor"] = team_season["ot_losses"] / (
            team_season["reg_losses"] + team_season["ot_losses"] + 1e-6)

# 4. Usage Normalization (Per 60)
# Assuming OT adds roughly 5 mins on average to a game
team_season["total_minutes"] = (team_season["games_played"] * 60) + (team_season["total_ot_games"] * 5)
team_season["xg_for_per_60"] = (team_season["xg_for"] / team_season["total_minutes"]) * 60
team_season["xg_against_per_60"] = (team_season["xg_against"] / team_season["total_minutes"]) * 60

team_season.sort_values("reg_win_pct", ascending=False)
# 1. Total wins (Reg + OT) for the standard Win %
team_games["total_win"] = (team_games["goal_diff"] > 0).astype(int)

# 2. Updated Aggregation
team_season = (
    team_games
    .groupby("team", as_index=False)
    .agg(
        games_played=("team", "count"),
        total_wins=("total_win", "sum"),  # For standard win_pct
        reg_wins=("reg_win", "sum"),  # For pure dominance
        ot_wins=("ot_win", "sum"),  # For luck factor
        ot_losses=("ot_loss", "sum"),  # For resilience
        reg_losses=("reg_loss", "sum"),  # For comparison
    )
)

# 3. Calculate the side-by-side rates
team_season["win_pct"] = team_season["total_wins"] / team_season["games_played"]
team_season["reg_win_pct"] = team_season["reg_wins"] / team_season["games_played"]

# 4. Calculate the 'Deception Gap'
team_season['deception_gap'] = team_season['win_pct'] - team_season['reg_win_pct']

# Display the comparison
print(team_season[['team', 'win_pct', 'reg_win_pct', 'deception_gap']].sort_values(by='deception_gap', ascending=False))

# Create the scatter plot
plot = sns.scatterplot(
    data=team_season,
    x='ot_reliance_ratio',
    y='resilience_factor',
    hue='reg_win_pct',
    size='games_played',
    palette='viridis',
    sizes=(50, 400)
)

# Add team names as labels
for i in range(team_season.shape[0]):
    plt.text(
        x=team_season.ot_reliance_ratio[i] + 0.005,
        y=team_season.resilience_factor[i] + 0.005,
        s=team_season.team[i],
        fontsize=9,
        alpha=0.7
    )

# Add quadrants for easier analysis
plt.axvline(team_season['ot_reliance_ratio'].mean(), color='red', linestyle='--', alpha=0.5)
plt.axhline(team_season['resilience_factor'].mean(), color='red', linestyle='--', alpha=0.5)

# Label the Quadrants
plt.text(0.4, 0.8, "Resilient Underdogs", color='gray', fontsize=12)
plt.text(0.4, 0.1, "Paper Tigers", color='gray', fontsize=12)

plt.title("WHL Team Profiles: OT Reliance vs Resilience (Labeled)")
plt.xlabel("OT Reliance (OT Wins / Total Wins)")
plt.ylabel("Resilience Factor (OT Losses / Total Losses)")
plt.legend(bbox_to_anchor=(1.05, 1), loc=2)
plt.show()
# 1. Ensure the underlying team_games has everything
# (This assumes team_games already has goals_for, shots_for, xg_for from your earlier concat)

# 2. Re-run the Master Aggregation
team_season = (
    team_games
    .groupby("team", as_index=False)
    .agg(
        games_played=("team", "count"),
        total_wins=("total_win", "sum"),
        reg_wins=("reg_win", "sum"),
        ot_wins=("ot_win", "sum"),
        ot_losses=("ot_loss", "sum"),
        reg_losses=("reg_loss", "sum"),
        shots_for=("shots_for", "sum"),  # Added this back
        shots_against=("shots_against", "sum"),  # Added this back
        xg_for=("xg_for", "sum"),
        xg_against=("xg_against", "sum")
    )
)

# 3. Re-calculate the rates
team_season["win_pct"] = team_season["total_wins"] / team_season["games_played"]
team_season["reg_win_pct"] = team_season["reg_wins"] / team_season["games_played"]
team_season["deception_gap"] = team_season["win_pct"] - team_season["reg_win_pct"]
team_season["xg_diff"] = team_season["xg_for"] - team_season["xg_against"]

# 4. Now the Corsi (Shot Share) calculation will work!
team_season["corsi_for_pct"] = team_season["shots_for"] / (team_season["shots_for"] + team_season["shots_against"])
# 1. Calculate the League-Wide Benchmarks (The Limits)
corsi_mean = team_season["corsi_for_pct"].mean()
gap_mean = team_season["deception_gap"].mean()

# 2. Print the Threshold Report
print("--- WHL MODEL CLASSIFICATION LIMITS ---")
print(f"Corsi Mean (Possession Threshold): {corsi_mean:.4f}")
print(f"Deception Mean (Luck Threshold):     {gap_mean:.4f}")
print("-" * 40)


# 3. Define the Logic
def identify_profile(row):
    # Quadrant 1: Low Possession, High OT Luck
    if row["corsi_for_pct"] < corsi_mean and row["deception_gap"] > gap_mean:
        return "Double Fraud"
    # Quadrant 2: High Possession, Low OT Luck (Dominant)
    if row["corsi_for_pct"] > corsi_mean and row["deception_gap"] < gap_mean:
        return "Pure Juggernaut"
    # Quadrant 3: High Possession, High OT Luck (Can't finish in 60)
    if row["corsi_for_pct"] > corsi_mean and row["deception_gap"] > gap_mean:
        return "Finishing Problem"
    # Quadrant 4: Low Possession, Low OT Luck
    return "Underdog"


# 4. Apply and Display
team_season["team_profile"] = team_season.apply(identify_profile, axis=1)

# Display results sorted by Deception Gap to highlight the Frauds
analysis_cols = ['team', 'win_pct', 'reg_win_pct', 'deception_gap', 'corsi_for_pct', 'team_profile']

print("Success! Data aggregated. Preview of top Corsi teams:")
print(team_season[['team', 'corsi_for_pct', 'deception_gap', 'team_profile']].sort_values('corsi_for_pct',
                                                                                          ascending=True))
a  # --- LOGIC FOR AWAY COLUMNS (Road Resilience & System Strength) ---
# 1. Road Resilience Score: Aggregate team xG when playing as 'away_team'.
#    - Compare 'Away xG/60' vs 'Home xG/60'.
#    - ACTION: Teams with the smallest "Home-Road Gap" get a Reliability Bonus.
#      They are 'System-Strong' and play well regardless of environment.
#
# 2. Defensive Opponent-Adjustment: Use away_def_pairing to 'weight' offensive success.
#    - Scoring against an opponent's 'first_def' is worth more Power Points
#      than scoring against their 'second_def'.
#    - ACTION: Create a 'Difficulty-Adjusted Goal' metric.
#
# 3. The "Last Change" Penalty: On the road, teams cannot control line matchups.
#    - If a team's 'away_off_line' (1st) still dominates while being 'hunted'
#      by the home coach, they are a Top-Tier Juggernaut.
#
# 4. Data Normalization: Combine Home and Away stats into a single 'Neutral Table'.
#    - This ensures a team's Power Ranking is based on their WHOLE season,
#      not just a favorable home schedule.

# --- LOGIC FOR GOALIE COLUMNS (The Gatekeeper Factor) ---
# 1. GSAx (Goals Saved Above Expected): Compare 'Actual Goals Allowed' vs 'Total xG'.
#    - Formula: GSAx = Total xG - Actual Goals.
#    - ACTION: High GSAx = Elite Goalie. Low GSAx = Weak Link.
#
# 2. Standings vs. Process: Identify teams 'carried' by their goalie.
#    - If a team wins despite being out-shot (low xG share), their rank is FRAGILE.
#    - ACTION: Weight 'Team xG' higher than 'Actual Wins' to find sustainable power.
#
# 3. Goalie Split: Check if a team has a clear 'Starter' vs 'Backup'.
#    - Does the team's Win % drop significantly when the backup goalie is in?
#    - ACTION: Create a 'Roster Reliability' score based on the gap between goalies.
#
# 4. The 'Sieve' Alert: Flag teams with high xG suppression (great defense)
#    but high Goals Against (bad goalie). These are 'Sleepers' if they swap goalies.
# --- LOGIC FOR TOI (The Normalization Key) ---
# 1. Rate Normalization: NEVER use raw xG or Goals for rankings.
#    Always calculate (Stat / toi) * 3600 to get the 'Per 60' rate.
#
# 2. Fatigue Analysis: Track total TOI for 'first_def' and 'first_off'.
#    Teams with extreme workloads for top units should get a 'Sustainability Penalty'
#    in long-term power rankings.
#
# 3. Small Sample Filter: Ignore or down-weight lines with less than
#    a certain threshold of total TOI (e.g., 500 seconds) to avoid 'fluke' stats.
#
# 4. Efficiency Mapping: Combine TOI with xG to see which lines are the
#    most 'lethal' per minute played.
# --- LOGIC FOR SHOTS & ASSISTS (Style & Luck Filter) ---
# 1. Shooting Percentage: (home_goals / home_shots).
#    - Identify 'Sustainability': If a team's shooting % is way above the league average,
#      expect their Power Ranking to drop later (regression).
#
# 2. Playmaking Grade: (home_assists / home_goals).
#    - High ratios indicate 'System Teams' with high puck movement.
#    - Low ratios indicate 'Individualist Teams' (reliant on solo efforts/turnovers).
#
# 3. Chaos Generator: Identify teams with high 'Shots per 60' but low xG.
#    - These teams play a 'dirty' game—relying on rebounds and volume rather than skill.
#
# 4. Assist Map: Link assists to off_line.
#    - Does the 1st line rely on assists while the 2nd line scores solo?
#    - ACTION: Use this to determine which line is easier to 'scout' and shut down.
# --- LOGIC FOR PENALTIES (The Discipline & Chaos Metric) ---
# 1. Discipline Rating: Calculate 'Penalty Minutes per 60'.
#    - Identify teams that 'beat themselves'. A high-penalizing team
#      should have their Power Score docked for 'Unreliability'.
#
# 2. Special Teams Exposure: Compare 'home_penalties' vs 'away_penalties'.
#    - If away_penalties >> home_penalties, the team is mentally fragile on the road.
#
# 3. Penalty-Adjusted xG: Create a 'Clean-Play xG' by filtering out records
#    where home_penalty == 1. This shows how good a team is when playing fair.
#
# 4. The 'Instigator' Factor: Does a team draw more penalties than they take?
#    - Compare 'home_penalties' vs 'away_penalties' in the same game.
#    - Teams that 'draw' penalties have high 'Functional Aggression'.
--- LOGIC
FOR
went_ot(The
Volatility
Filter) ---
# 1. Regulation Performance: Use this to isolate 'Regulation Goal Differential'.
#    A team winning 5-0 in regulation is significantly stronger than a team
#    winning 1-0 in OT. The former shows dominance; the latter shows a coin-flip.
#
# 2. 'The Paper Tiger' Check: Identify teams with high standings but high OT win rates.
#    If a team relies on OT/Shootouts, their Power Ranking should be ADJUSTED DOWN
#    as OT results are less repeatable than 5-on-5 play.
#
# 3. 'The Resilience' Factor: Boost teams with high OT Loss counts.
#    In the standings, they look like losers (0 wins), but in reality,
#    they are competitive enough to hold elite teams to a draw for 60 minutes.
#
# 4. Usage Normalization: Since OT adds extra 'toi', always use 'per 60 minutes'
#    rates (e.g., xG/60) to ensure OT minutes don't artificially inflate total stats.

--- LOGIC
FOR
home_off_line(The
Roster
Strength
Factor) ---
# 1. Roster Depth: Compare 'first_off' vs 'second_off' xG/60.
#    - 'One-Line Wonders': Teams with a huge drop-off in quality (e.g., 1st line 3.0 xG, 2nd line 0.5 xG).
#    - 'Balanced Giants': Teams where both lines produce consistently.
#    ACTION: Reward 'Balanced' teams with a higher stability score in rankings.
#
# 2. Situational Power: Isolate 'PP_up' (Power Play) records.
#    - Standing might be low, but if 'PP_up' xG/60 is top 5, they are a 'Danger Team'.
#    ACTION: Add a 'Special Teams Grade' to the final Power Ranking.
#
# 3. 5-on-5 Purity: Filter for 'first_off' and 'second_off' only to find 'Even-Strength' dominance.
#    - This is the most repeatable part of hockey.
#    ACTION: Use Even-Strength xG Differential as 50% of the total Power Ranking weight.
#
# 4. Tactical Matchups: Link with 'away_def_pairing' to see which lines 'crush' weaker defenders.
#    - Identify teams that successfully hunt mismatches (e.g., first_off vs. opponent's second_def).
# --- LOGIC FOR AWAY COLUMNS (Road Resilience & System Strength) ---
# 1. Road Resilience Score: Aggregate team xG when playing as 'away_team'.
#    - Compare 'Away xG/60' vs 'Home xG/60'.
#    - ACTION: Teams with the smallest "Home-Road Gap" get a Reliability Bonus.
#      They are 'System-Strong' and play well regardless of environment.
#
# 2. Defensive Opponent-Adjustment: Use away_def_pairing to 'weight' offensive success.
#    - Scoring against an opponent's 'first_def' is worth more Power Points
#      than scoring against their 'second_def'.
#    - ACTION: Create a 'Difficulty-Adjusted Goal' metric.
#
# 3. The "Last Change" Penalty: On the road, teams cannot control line matchups.
#    - If a team's 'away_off_line' (1st) still dominates while being 'hunted'
#      by the home coach, they are a Top-Tier Juggernaut.
#
# 4. Data Normalization: Combine Home and Away stats into a single 'Neutral Table'.
#    - This ensures a team's Power Ranking is based on their WHOLE season,
#      not just a favorable home schedule.
# 1. Calculate xG/60 for each offensive line
line_stats = df.groupby(['home_team', 'home_off_line']).agg(
    total_xg=('home_xg', 'sum'),
    total_toi=('toi', 'sum')
).reset_index()

line_stats['xg_60'] = (line_stats['total_xg'] / line_stats['total_toi']) * 60

# 2. Pivot to compare lines
roster_pivot = line_stats.pivot(index='home_team', columns='home_off_line', values='xg_60')

# 3. Calculate 'Purity' (Average of top 2 lines) and 'Stability' (The Gap)
roster_pivot['five_on_five_purity'] = (roster_pivot['first_off'] + roster_pivot['second_off']) / 2
roster_pivot['roster_stability_ratio'] = roster_pivot['first_off'] / (roster_pivot['second_off'] + 1e-6)

print("--- PILLAR 1: 5-on-5 PURITY & STABILITY ---")
print(roster_pivot[['five_on_five_purity', 'roster_stability_ratio']].sort_values('five_on_five_purity',
                                                                                  ascending=False).head())
# --- PRETEXT: AUTO-CALIBRATING ROSTER DEPTH ---
# Instead of guessing that a 1.5 ratio is 'Top Heavy', this code
# looks at the entire league's distribution and labels the
# outliers. This ensures we ALWAYS find the most top-heavy teams.

# 1. Calculate the actual median ratio for the league
median_stability = roster_pivot['roster_stability_ratio'].median()
median_purity = roster_pivot['five_on_five_purity'].median()

print(f"League Average Stability Ratio: {median_stability:.2f}")


# 2. Re-classify using Relative Thresholds
def classify_offense_dynamic(row):
    # Above average production?
    is_high_scoring = row['five_on_five_purity'] > median_purity
    # More top-heavy than the average team?
    is_top_heavy = row['roster_stability_ratio'] > median_stability

    if is_high_scoring:
        return "Balanced Giant" if not is_top_heavy else "One-Line Wonder"
    else:
        return "Deep but Weak" if not is_top_heavy else "Top-Heavy Underdog"


roster_pivot['off_identity'] = roster_pivot.apply(classify_offense_dynamic, axis=1)

# 3. Check the distribution
print(roster_pivot['off_identity'].value_counts())

# 1. Calculate medians for dynamic labeling
# We use the median so that exactly half the league is 'High Scoring'
# and half is 'Top Heavy' (relatively speaking).
median_purity = roster_pivot['five_on_five_purity'].median()
median_stability = roster_pivot['roster_stability_ratio'].median()


# 2. Engineering the Label Column
def classify_offense_dynamic(row):
    # Above average production?
    is_high_scoring = row['five_on_five_purity'] > median_purity
    # More top-heavy than the average team?
    is_top_heavy = row['roster_stability_ratio'] > median_stability

    if is_high_scoring:
        # High scoring + Balanced depth
        if not is_top_heavy:
            return "Balanced Giant"
        # High scoring + Reliance on 1st line
        else:
            return "One-Line Wonder"
    else:
        # Low scoring + Balanced depth
        if not is_top_heavy:
            return "Deep but Weak"
        # Low scoring + Reliance on 1st line
        else:
            return "Top-Heavy Underdog"


# Apply the labels to a new feature column
roster_pivot['off_identity'] = roster_pivot.apply(classify_offense_dynamic, axis=1)

# 3. Print the engineered feature set
print("--- OFFENSIVE FEATURE ENGINEERING: TEXT LABELS ---")
print(roster_pivot[['five_on_five_purity', 'roster_stability_ratio', 'off_identity']].sort_values('five_on_five_purity',
                                                                                                  ascending=False))
# --- FEATURE ENGINEERING: DEFENSIVE & GOALIE IDENTITY ---

# 1. Aggregate Defensive Data
# We focus on 'first_def' as they face the highest quality competition
def_stats = df.groupby(['home_team', 'home_def_pairing']).agg(
    xg_against=('away_xg', 'sum'),
    goals_against=('away_goals', 'sum'),
    total_toi=('toi', 'sum')
).reset_index()

# 2. Calculate Normalized Rates
def_stats['xGA_60'] = (def_stats['xg_against'] / def_stats['total_toi']) * 60
def_stats['actual_GA_60'] = (def_stats['goals_against'] / def_stats['total_toi']) * 60

# 3. Engineer the 'Goalie Factor'
# (Actual - Expected). Negative means the goalie saved more than expected.
def_stats['goalie_factor'] = def_stats['actual_GA_60'] - def_stats['xGA_60']

# 4. Set Thresholds for Labeling
shutdown_unit = def_stats[def_stats['home_def_pairing'] == 'first_def'].set_index('home_team')
median_xGA = shutdown_unit['xGA_60'].median()


# 5. Engineering the Text Label Column
def classify_goalie_performance(row):
    # System Check
    is_steel_wall = row['xGA_60'] < median_xGA

    # Goalie Check
    if row['goalie_factor'] < -0.2:
        goalie_label = "Goalie Hero"
    elif row['goalie_factor'] > 0.2:
        goalie_label = "Goalie Vulnerable"
    else:
        goalie_label = "Standard Support"

    system_label = "Steel Wall" if is_steel_wall else "Leaky System"
    return f"{system_label} ({goalie_label})"


shutdown_unit['def_identity'] = shutdown_unit.apply(classify_goalie_performance, axis=1)

# 6. Display the Results
print("--- DEFENSIVE FEATURE ENGINEERING: GOALIE LABELS ---")
print(shutdown_unit[['xGA_60', 'goalie_factor', 'def_identity']].sort_values('goalie_factor'))
# --- FEATURE ENGINEERING: THE TEAM DNA SYNTHESIS ---

# 1. Merge the Offensive and Defensive identities
# We join the two pivoted/engineered dataframes on the team name
team_dna_report = roster_pivot[['off_identity', 'five_on_five_purity']].merge(
    shutdown_unit[['def_identity', 'xGA_60', 'goalie_factor']],
    left_index=True,
    right_index=True
)


# 2. Engineer the 'Strategic Archetype' Label
# This is a high-level feature for executive summaries
def label_archetype(row):
    if row['off_identity'] == "Balanced Giant" and "Steel Wall" in row['def_identity']:
        return "Complete Contender"
    elif row['off_identity'] == "One-Line Wonder" and "Goalie Hero" in row['def_identity']:
        return "The Glass House (Star-Dependent)"
    elif "Leaky System" in row['def_identity'] and row['off_identity'] == "One-Line Wonder":
        return "Critical Risk"
    elif "Steel Wall" in row['def_identity'] and row['off_identity'] == "Deep but Weak":
        return "The Defensive Specialist"
    else:
        return "The Wildcard"


team_dna_report['strategic_archetype'] = team_dna_report.apply(label_archetype, axis=1)

# 1. Merge Offensive Features and Defensive Features into one row per team
# We use 'inner' to ensure we only keep teams that have both offensive and defensive data
team_master_features = roster_pivot.merge(
    shutdown_unit,
    left_index=True,
    right_index=True,
    suffixes=('_off', '_def')
)

# 2. Select and Rename the most important engineered columns for clarity
final_features = team_master_features[[
    'off_identity',  # Balanced Giant, One-Line Wonder, etc.
    'def_identity',  # Steel Wall (Hero), Leaky (Vulnerable), etc.
    'five_on_five_purity',
    'xGA_60',
    'goalie_factor'
]]

# 3. Print the results - Now everything is on the SAME LINE
print("--- CONSOLIDATED TEAM FEATURES ---")
final_features.sort_values('five_on_five_purity', ascending=False)
# 1. Unifying the Pillars
# We join 'roster_pivot' (Offense) and 'shutdown_unit' (Defense)
# Since both are indexed by team name, they will align perfectly on the same line.
team_features_df = roster_pivot[['off_identity', 'five_on_five_purity', 'roster_stability_ratio']].join(
    shutdown_unit[['def_identity', 'xGA_60', 'goalie_factor']]
)

# 2. Engineering the 'Master Profile' Label
# This creates a single text string summarizing the whole team identity
team_features_df['team_profile'] = (
        team_features_df['off_identity'] + " | " + team_features_df['def_identity']
)

# 3. Handle any missing data (NaNs) just in case
team_features_df = team_features_df.fillna("Unknown/Incomplete")

# 4. Display the result in Jupyter
print("--- MASTER FEATURE MATRIX DEFINED ---")
display(team_features_df.head())
# 1. Aggregate Special Teams Performance
special_stats = df.groupby(['home_team', 'home_off_line']).agg(
    pp_xg=('home_xg', 'sum'),
    pp_toi=('toi', 'sum')
).reset_index()

# Filter for just the Power Play units
pp_units = special_stats[special_stats['home_off_line'] == 'PP_up'].copy()
pp_units['pp_xg_60'] = (pp_units['pp_xg'] / pp_units['pp_toi']) * 60

# 2. Join this back to your Master Matrix
team_features_df = team_features_df.join(pp_units.set_index('home_team')[['pp_xg_60']])

# 3. Labeling the Special Teams Specialist
pp_median = team_features_df['pp_xg_60'].median()


def label_special_teams(row):
    if row['pp_xg_60'] > team_features_df['pp_xg_60'].quantile(0.75):
        return "Special Teams Specialist"
    elif row['pp_xg_60'] < team_features_df['pp_xg_60'].quantile(0.25):
        return "Special Teams Liability"
    else:
        return "Standard Unit"


team_features_df['st_identity'] = team_features_df.apply(label_special_teams, axis=1)

# 4. Display the updated Master Matrix
team_features_df[['off_identity', 'def_identity', 'st_identity', 'pp_xg_60']].sort_values('pp_xg_60', ascending=False)

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import zscore, rankdata

# Load data
df = pd.read_excel('./datasci/cleanedup/whl_2025_base.xlsx')

# --- Data Processing (Team-Game Level) ---
home_df = df[['game_id', 'home_team', 'home_goals', 'away_goals', 'home_xg', 'away_xg', 'home_shots', 'away_shots',
              'home_penalty_minutes', 'went_ot']].copy()
home_df.columns = ['game_id', 'team', 'gf', 'ga', 'xgf', 'xga', 'sf', 'sa', 'pim', 'is_ot']
home_df['is_home'] = 1
# Simple Points Calculation (2 for win, 1 for OT loss? We'll assume Win=2)
home_df['pts'] = np.where(home_df['gf'] > home_df['ga'], 2, np.where(home_df['is_ot'] == 1, 1, 0))

away_df = df[['game_id', 'away_team', 'away_goals', 'home_goals', 'away_xg', 'home_xg', 'away_shots', 'home_shots',
              'away_penalty_minutes', 'went_ot']].copy()
away_df.columns = ['game_id', 'team', 'gf', 'ga', 'xgf', 'xga', 'sf', 'sa', 'pim', 'is_ot']
away_df['is_home'] = 0
away_df['pts'] = np.where(away_df['gf'] > away_df['ga'], 2, np.where(away_df['is_ot'] == 1, 1, 0))

team_games = pd.concat([home_df, away_df], ignore_index=True)

# --- Aggregations & Metrics ---
stats = team_games.groupby('team').agg(
    gp=('game_id', 'count'),
    gf=('gf', 'sum'),
    ga=('ga', 'sum'),
    xgf=('xgf', 'sum'),
    xga=('xga', 'sum'),
    pts=('pts', 'sum'),
    xga_var=('xga', 'var')  # Variance of Expected Goals Against (Consistency Proxy)
).reset_index()

# 1. The Shutdown Metric (Defense)
stats['xGA_per_GP'] = stats['xga'] / stats['gp']
stats['Def_Consistency'] = stats['xga_var']  # Lower is better

# 2. The Gatekeeper Factor (Goaltending)
stats['GSAx'] = stats['xga'] - stats['ga']
stats['PTS_Pct'] = stats['pts'] / (stats['gp'] * 2)
stats['xG_Share'] = stats['xgf'] / (stats['xgf'] + stats['xga'])
stats['Sustain_Diff'] = stats['PTS_Pct'] - stats['xG_Share']


# Sustain_Diff > 0: Overperforming (Goalie Saved Them?)
# Sustain_Diff < 0: Underperforming (Goalied?)

# --- Normalization & Ranking ---
def normalize(series, invert=False):
    if invert:
        return 1 - ((series - series.min()) / (series.max() - series.min()))
    return (series - series.min()) / (series.max() - series.min())


# Defense Score (Higher is better)
# Low xGA is good (Invert)
# Low Variance is good (Invert)
stats['Score_Def'] = (normalize(stats['xGA_per_GP'], invert=True) * 0.7) + (
            normalize(stats['Def_Consistency'], invert=True) * 0.3)

# Goalie Score (Higher is better)
# High GSAx is good
# High Sustain Diff? Actually we want sustainable power.
# The user said: "High GSAx = Elite". "If win despite being outshot... Rank is FRAGILE".
# So we reward GSAx, but maybe penalize extreme reliance on it?
# Request: "Weight Team xG higher than Actual Wins".
# Let's stick to GSAx as the core "Gatekeeper" quality metric.
stats['Score_Goalie'] = normalize(stats['GSAx'])

# Final Ranking (50/50 Split)
stats['PowerScore'] = (stats['Score_Def'] * 0.5 + stats['Score_Goalie'] * 0.5) * 100
stats['Rank'] = stats['PowerScore'].rank(ascending=False).astype(int)
stats = stats.sort_values('Rank')

# --- The "Sieve" Alert Logic ---
# Top 33% Defense (Low xGA) AND Bottom 33% Goaltending (Low GSAx)
xga_threshold = stats['xGA_per_GP'].quantile(0.33)
gsax_threshold = stats['GSAx'].quantile(0.33)


def sieves(row):
    if row['xGA_per_GP'] <= xga_threshold and row['GSAx'] <= gsax_threshold:
        return 'SIEVE ALERT (Good Def/Bad Goalie)'
    if row['GSAx'] >= stats['GSAx'].quantile(0.90):
        return 'ELITE GOALIE'
    if row['xGA_per_GP'] <= xga_threshold:
        return 'ELITE DEFENSE'
    return '-'


stats['Status'] = stats.apply(sieves, axis=1)

# --- Output ---
cols = ['Rank', 'team', 'PowerScore', 'Status', 'xGA_per_GP', 'GSAx', 'Sustain_Diff']
print(stats[cols].to_string(index=False, float_format='%.2f'))

# --- Visuals ---
plt.style.use('dark_background')
plt.figure(figsize=(12, 8))

# Quadrant Plot: Defense Quality (xGA/GP) vs Goalie Performance (GSAx)
# Note: xGA/GP is better when LOWER. So we invert X axis or interpret left as good.
sns.scatterplot(
    data=stats,
    x='xGA_per_GP',
    y='GSAx',
    hue='Status',
    style='Status',
    s=150,
    palette='viridis'
)

# Invert X axis so Right = Better Defense? Or Left = Better?
# Standard is Left = Lower xGA = Better.
plt.gca().invert_xaxis()

for i, row in stats.iterrows():
    plt.text(row['xGA_per_GP'], row['GSAx'] + 0.5, row['team'], fontsize=8, color='white')

plt.title('The Shutdown & Gatekeeper Map', fontsize=16)
plt.xlabel('Defense Quality (xGA per Game) --> Better', fontsize=12)
plt.ylabel('Goalie Performance (GSAx) Better -->', fontsize=12)
plt.axvline(stats['xGA_per_GP'].mean(), color='gray', linestyle='--')
plt.axhline(stats['GSAx'].mean(), color='gray', linestyle='--')
plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
plt.tight_layout()
plt.show()

# --- LOGIC FOR home_def_pairing (The Shutdown Metric) ---
# 1. Shutdown Quality: Calculate 'xG Allowed per 60' for each pairing.
#    A team's 'Defensive Rank' should be heavily weighted by the first_def unit.
#
# 2. Defensive Depth: Measure the 'Reliability Gap' between 1st and 2nd pairs.
#    Teams with a strong second_def are 'Tournament Hardened' and harder to exploit.
#
# 3. PK Specialist Rank: Filter for 'PP_kill_dwn'.
#    Identify teams that effectively suppress xG even when man-down.
#    High PK efficiency is a major signal for 'Playoff Ready' power rankings.
#
# 4. Goal-Save Delta: Compare 'Actual Goals Allowed' vs 'xG Allowed' per pairing.
#    If a pairing allows high xG but zero goals, the goalie is 'bailing them out'.
# --- LOGIC FOR GOALIE COLUMNS (The Gatekeeper Factor) ---
# 1. GSAx (Goals Saved Above Expected): Compare 'Actual Goals Allowed' vs 'Total xG'.
#    - Formula: GSAx = Total xG - Actual Goals.
#    - ACTION: High GSAx = Elite Goalie. Low GSAx = Weak Link.
#
# 2. Standings vs. Process: Identify teams 'carried' by their goalie.
#    - If a team wins despite being out-shot (low xG share), their rank is FRAGILE.
#    - ACTION: Weight 'Team xG' higher than 'Actual Wins' to find sustainable power.
#
# 3. Goalie Split: Check if a team has a clear 'Starter' vs 'Backup'.
#    - Does the team's Win % drop significantly when the backup goalie is in?
#    - ACTION: Create a 'Roster Reliability' score based on the gap between goalies.
FREAKY
DIDDY
ARMAAN
GUHA
# 4. The 'Sieve' Alert: Flag teams with high xG suppression (great defense)
#    but high Goals Against (bad goalie). These are 'Sleepers' if they swap goalies.,
#      not just a favorable home schedule.

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf

df = pd.DataFrame(pd.read_excel('./datasci/cleanedup/whl_2025_base.xlsx'))
df.head()

'''
IDENTIFIERS:
- game_id
- record_id

ENTITIES:
- home_team
- away_team
- home_goalie
- away_goalie

CONTEXT:
- home_off_line
- away_off_line
- home_def_pairing
- away_def_pairing
- went_ot

OUTCOMES:
- home_goals
- away_goals
- home_shots
- away_shots
- home_penalties_committed
- away_penalties_committed

DERIVED METRICS:
- home_xg
- away_xg
- home_max_xg
- away_max_xg
'''
sum_cols = [
    "home_goals", "away_goals",
    "home_shots", "away_shots",
    "home_xg", "away_xg",
    "home_assists", "away_assists",
    "home_penalties_committed", "away_penalties_committed",
    "home_penalty_minutes", "away_penalty_minutes",
    "toi",
]
first_cols = [
    "home_team",
    "away_team",
    "went_ot"
]
agg_dict = {}

for col in sum_cols:
    agg_dict[col] = "sum"

for col in first_cols:
    agg_dict[col] = "first"
games = (
    df
    .groupby("game_id", as_index=False)
    .agg(agg_dict)
)
games.shape
games["home_score"] = games["home_goals"]
games["away_score"] = games["away_goals"]

games["goal_diff"] = games["home_score"] - games["away_score"]
games["total_goals"] = games["home_score"] + games["away_score"]

games["shot_diff"] = games["home_shots"] - games["away_shots"]
games["total_shots"] = games["home_shots"] + games["away_shots"]
games["total_toi"] = games["toi"]

games

home_games = games.copy()

home_games["team"] = home_games["home_team"]
home_games["opponent"] = home_games["away_team"]

home_games["goals_for"] = home_games["home_score"]
home_games["goals_against"] = home_games["away_score"]

home_games["shots_for"] = home_games["home_shots"]
home_games["shots_against"] = home_games["away_shots"]

home_games["xg_for"] = home_games["home_xg"]
home_games["xg_against"] = home_games["away_xg"]

home_games["is_home"] = 1
away_games = games.copy()

away_games["team"] = away_games["away_team"]
away_games["opponent"] = away_games["home_team"]

away_games["goals_for"] = away_games["away_score"]
away_games["goals_against"] = away_games["home_score"]

away_games["shots_for"] = away_games["away_shots"]
away_games["shots_against"] = away_games["home_shots"]

away_games["xg_for"] = away_games["away_xg"]
away_games["xg_against"] = away_games["home_xg"]
away_games["is_home"] = 0

# --- LOGIC FOR TOI (The Normalization Key) ---
# 1. Rate Normalization: NEVER use raw xG or Goals for rankings.
#    Always calculate (Stat / toi) * 3600 to get the 'Per 60' rate.
#
# 2. Fatigue Analysis: Track total TOI for 'first_def' and 'first_off'.
#    Teams with extreme workloads for top units should get a 'Sustainability Penalty'
#    in long-term power rankings.
#
# 3. Small Sample Filter: Ignore or down-weight lines with less than
#    a certain threshold of total TOI (e.g., 500 seconds) to avoid 'fluke' stats.
#
# 4. Efficiency Mapping: Combine TOI with xG to see which lines are the
#    most 'lethal' per minute played.
# --- LOGIC FOR SHOTS & ASSISTS (Style & Luck Filter) ---
# 1. Shooting Percentage: (home_goals / home_shots).
#    - Identify 'Sustainability': If a team's shooting % is way above the league average,
#      expect their Power Ranking to drop later (regression).
#
# 2. Playmaking Grade: (home_assists / home_goals).
#    - High ratios indicate 'System Teams' with high puck movement.
#    - Low ratios indicate 'Individualist Teams' (reliant on solo efforts/turnovers).
#
# 3. Chaos Generator: Identify teams with high 'Shots per 60' but low xG.
#    - These teams play a 'dirty' game—relying on rebounds and volume rather than skill.
#
# 4. Assist Map: Link assists to off_line.
#    - Does the 1st line rely on assists while the 2nd line scores solo?
#    - ACTION: Use this to determine which line is easier to 'scout' and shut down.
# --- LOGIC FOR PENALTIES (The Discipline & Chaos Metric) ---
# 1. Discipline Rating: Calculate 'Penalty Minutes per 60'.
#    - Identify teams that 'beat themselves'. A high-penalizing team
#      should have their Power Score docked for 'Unreliability'.
#
# 2. Special Teams Exposure: Compare 'home_penalties' vs 'away_penalties'.
#    - If away_penalties >> home_penalties, the team is mentally fragile on the road.
#
# 3. Penalty-Adjusted xG: Create a 'Clean-Play xG' by filtering out records
#    where home_penalty == 1. This shows how good a team is when playing fair.
#
# 4. The 'Instigator' Factor: Does a team draw more penalties than they take?
#    - Compare 'home_penalties' vs 'away_penalties' in the same game.
#    - Teams that 'draw' penalties have high 'Functional Aggression'.
team_games = pd.concat([home_games, away_games], ignore_index=True)
team_games["goal_diff"] = team_games["goals_for"] - team_games["goals_against"]

team_games["win"] = (team_games["goal_diff"] > 0).astype(int)
team_games["loss"] = (team_games["goal_diff"] < 0).astype(int)
team_season = (
    team_games
    .groupby("team", as_index=False)
    .agg(
        games_played=("team", "count"),
        wins=("win", "sum"),
        losses=("loss", "sum"),
        goals_for=("goals_for", "sum"),
        goals_against=("goals_against", "sum"),
        shots_for=("shots_for", "sum"),
        shots_against=("shots_against", "sum"),
        xg_for=("xg_for", "sum"),
        xg_against=("xg_against", "sum"),
        avg_goal_diff=("goal_diff", "mean"),
        home_games=("is_home", "sum"),
        toi=("toi", "sum"),
    ))
team_season["goals_per_game"] = team_season["goals_for"] / team_season["games_played"]
team_season["goals_against_per_game"] = team_season["goals_against"] / team_season["games_played"]

team_season["shot_diff"] = team_season["shots_for"] - team_season["shots_against"]
team_season["xg_diff"] = team_season["xg_for"] - team_season["xg_against"]
team_season["win_pct"] = team_season["wins"] / team_season["games_played"]
team_season["loss_pct"] = team_season["losses"] / team_season["games_played"]
team_season

# Home offensive line TOI
home_line_toi = (
    df
    .groupby(["home_team", "home_off_line"], as_index=False)
    .agg(
        total_toi=("toi", "sum"),
        games=("game_id", "nunique")
    )
    .rename(columns={
        "home_team": "team",
        "home_off_line": "off_line"
    })
)

home_line_toi["side"] = "home"

# Away offensive line TOI
away_line_toi = (
    df
    .groupby(["away_team", "away_off_line"], as_index=False)
    .agg(
        total_toi=("toi", "sum"),
        games=("game_id", "nunique")
    )
    .rename(columns={
        "away_team": "team",
        "away_off_line": "off_line"
    })
)

away_line_toi["side"] = "away"

line_toi = pd.concat(
    [home_line_toi, away_line_toi],
    ignore_index=True
)
line_toi_total = (
    line_toi
    .groupby(["team", "off_line"], as_index=False)
    .agg(
        total_toi=("total_toi", "sum"),
        games=("games", "sum")
    )
)
print(line_toi_total.head(128))

home_lines = (
    df
    .groupby(["home_team", "home_off_line"], as_index=False)
    .agg(
        toi=("toi", "sum"),
        xg=("home_xg", "sum"),
        goals=("home_goals", "sum"),
        games=("game_id", "nunique")
    )
    .rename(columns={
        "home_team": "team",
        "home_off_line": "off_line"
    })
)

away_lines = (
    df
    .groupby(["away_team", "away_off_line"], as_index=False)
    .agg(
        toi=("toi", "sum"),
        xg=("away_xg", "sum"),
        goals=("away_goals", "sum"),
        games=("game_id", "nunique")
    )
    .rename(columns={
        "away_team": "team",
        "away_off_line": "off_line"
    })
)

off_lines = pd.concat([home_lines, away_lines], ignore_index=True)

off_lines = off_lines[off_lines["toi"] >= 500].copy()

off_lines["xg_60"] = off_lines["xg"] / off_lines["toi"] * 3600
off_lines["goals_60"] = off_lines["goals"] / off_lines["toi"] * 3600  # Get team wins from team_season
team_wins = team_season[["team", "wins"]]
off_lines = off_lines.merge(team_wins, on="team", how="left")

off_lines["efficiency"] = off_lines["xg"] / off_lines["toi"]

team_toi = (
    off_lines
    .groupby("team", as_index=False)
    .agg(team_toi=("toi", "sum"))
)

off_lines = off_lines.merge(team_toi, on="team", how="left")

off_lines["fatigue_ratio"] = off_lines["toi"] / off_lines["team_toi"]

off_lines["overworked"] = off_lines["fatigue_ratio"] > 0.185

off_lines = off_lines.sort_values("goals", ascending=False)
print(off_lines.head(32))
print(off_lines["fatigue_ratio"].describe())

# Check the actual values before the boolean check
print(off_lines[['team', 'off_line', 'toi', 'team_toi', 'fatigue_ratio']].head(10))

# Try a dynamic threshold instead of a hard 0.20
# This identifies the 'top 25%' most used lines in the league
fatigue_threshold = off_lines["fatigue_ratio"].quantile(0.75)
print(f"Top 25% Fatigue Threshold: {fatigue_threshold:.4f}")

off_lines["overworked"] = off_lines["fatigue_ratio"] > fatigue_threshold
# 1. Use the Dynamic Threshold (Top Quartile of the actual data)
# This finds the 'Workhorses' relative to the rest of the league
league_fatigue_cutoff = off_lines["fatigue_ratio"].quantile(0.75)

# 2. Re-calculate the overworked flag
off_lines["overworked"] = off_lines["fatigue_ratio"] > league_fatigue_cutoff

# 3. Validation: Check if we now have 'True' values
overworked_count = off_lines["overworked"].sum()

print(f"League-Wide Fatigue Cutoff: {league_fatigue_cutoff:.4f}")
print(f"Number of 'Overworked' lines identified: {overworked_count}")

# 4. Display the 'Top 10' busiest lines in the league
print("\n--- TOP 10 BUSIEST LINES (RELATIVE FATIGUE) ---")
display(off_lines[['team', 'off_line', 'fatigue_ratio', 'overworked']].head(10))
# 1. Filter for only the 'Overworked' lines
fatigue_report = off_lines[off_lines["overworked"] == True].copy()

# 2. Sort by the highest fatigue ratio
fatigue_report = fatigue_report.sort_values("fatigue_ratio", ascending=False)

# 3. Clean up the columns for a 'scouting' look
# We'll show the Team, the specific Line, and how much of the team's time they eat
fatigue_report["Usage %"] = (fatigue_report["fatigue_ratio"] * 100).round(2).astype(str) + "%"
# 1. Create a clean text label based on the overworked flag
off_lines["workload_status"] = off_lines["overworked"].map({
    True: "FATIGUED",
    False: "FRESH",
})

# 2. Display the final scouting table
# We'll filter for a mix of both so you can see the labels in action
# 1. Filter the scouting table for only the second offensive lines
second_off_report = scouting_table[scouting_table["off_line"] == "second_off"].copy()

# 2. Display the filtered table
print("--- SCOUTING REPORT: SECOND LINE WORKLOAD ---")
display(second_off_report[['team', 'off_line', 'fatigue_ratio', 'workload_status']])

# 4. Display the top 20 most overworked lines
print("--- THE FATIGUE WATCHLIST: TOP 20 WORKHORSE LINES ---")
display(fatigue_report[['team', 'off_line', 'Usage %', 'xg_60', 'games']])


In [ ]:
# --- PENALTY FEATURE ENGINEERING ---

# 1. Calculate Taken vs Drawn PIM
# PIM Taken = Home team penalties
# PIM Drawn = Away team penalties (while playing at home)
penalty_base = df.groupby('home_team').agg(
    pim_taken=('home_penalty_minutes', 'sum'),
    pim_drawn=('away_penalty_minutes', 'sum'),
    total_toi=('toi', 'sum')
).reset_index()

# 2. Normalize to 'Per 60' and 'Net'
penalty_base['pim_taken_60'] = (penalty_base['pim_taken'] / (penalty_base['total_toi'] / 3600))
penalty_base['net_pim'] = penalty_base['pim_drawn'] - penalty_base['pim_taken']

# 3. Define Thresholds (League Medians)
median_pim_60 = penalty_base['pim_taken_60'].median()
median_net = penalty_base['net_pim'].median()

# 4. The Classification Logic (The "Four Quadrants")
def classify_penalties(row):
    is_high_risk = row['pim_taken_60'] > median_pim_60
    is_positive_net = row['net_pim'] > median_net

    if is_high_risk and not is_positive_net:
        return "The Liability (High Risk / Net Loss)"
    elif is_high_risk and is_positive_net:
        return "The Instigator (High Risk / Net Gain)"
    elif not is_high_risk and is_positive_net:
        return "The Professional (Disciplined / Net Gain)"
    else:
        return "The Passive (Disciplined / Net Loss)"

penalty_base['penalty_identity'] = penalty_base.apply(classify_penalties, axis=1)

# 5. Merge into your Master Matrix
team_features_df = team_features_df.join(penalty_base.set_index('home_team')[['pim_taken_60', 'net_pim', 'penalty_identity']])

# --- DISPLAY THE "SINNER" LIST ---
print("--- WHL PENALTY & RISK ANALYSIS ---")
display(team_features_df[['penalty_identity', 'pim_taken_60', 'net_pim']].sort_values('pim_taken_60', ascending=False))